In [30]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
from joblib import Parallel, delayed


import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip3 install spacy==3.4

In [ ]:
#!python3 -m spacy download en_core_web_md

In [2]:
coffee = pd.read_csv('../data/coffee_fix_clean.csv')

In [3]:
coffee = coffee.loc[:,['name','roaster','roaster_city','roaster_country_state','roaster_country','origin',
                       'origin_count','origin_country','review_date','est_price','rating','roast','agtron',
                       'agtron_grounds','agtron_bean','aroma','acid','body','flavor','aftertaste','with_milk',
                       'desc_1','desc_2','desc_3']]

In [65]:
coffee.to_csv('../data/coffee_final.csv',index=False)

In [4]:
origin_feature=pd.get_dummies(coffee['origin_country'])

In [5]:
origin_feature

,Africa,Africa; Asia,Africa; Asia Pacific,Africa; Central America,Africa; Indonesia; South America,Africa; South America,Africa; South America; Central America,America; Indo-Pacific,Asia; Africa; America,Bolivia,...,South America,Taiwan,Tanzania,Thailand,Thailand; Laos,Thailand; Vietnam; Latin America,Uganda,Vietnam,Yemen,Zambia
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2273,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
neighbour_origin = NearestNeighbors()

In [ ]:
user_input = 'Colombia'

In [ ]:
neighbour_origin.fit(origin_feature.loc[:,[user_input]])

In [ ]:
index = neighbour_origin.kneighbors([[1]], 2, return_distance=False)

In [ ]:
select_index = np.reshape(index,-1)

In [ ]:
coffee.iloc[select_index,:]

In [ ]:
taste_feature = coffee.loc[:,['aroma', 'acid', 'body', 'flavor', 'aftertaste']]

In [ ]:
taste_feature

In [ ]:
neighbour_taste = NearestNeighbors()

In [ ]:
neighbour_taste.fit(taste_feature)

In [ ]:
taste_input = [[5,2,8,5,9]]

In [ ]:
taste_index = neighbour_taste.kneighbors(taste_input, 2, return_distance=False)

In [ ]:
select_taste_index = np.reshape(taste_index,-1)

In [ ]:
coffee.iloc[select_taste_index,:]

In [12]:
coffee['desc_3'] = coffee['desc_3'].fillna('')

In [13]:
coffee['desc'] = coffee['desc_1'] + ' ' + coffee['desc_3']

In [14]:
coffee.iloc[0][24]

'Evaluated as espresso. Sweet-toned, deeply rich, chocolaty. Vanilla paste, dark chocolate, narcissus, pink grapefruit zest, black cherry in aroma and cup. Plush, syrupy mouthfeel; resonant, flavor-saturated finish. In three parts milk, rich chocolate tones intensify, along with intimations of vanilla paste and black cherry in the short finish and floral-toned citrus zest in the long.  A radiant espresso blend that shines equally in the straight shot and in milk, alive with notes of rich dark chocolate and black cherry.'

In [91]:
country_list = []
for country_row in coffee.origin_country.str.split(';'):
    for country in country_row:
        country = country.strip()
        country_list.append(country)

In [95]:
country_list = list(set(country_list))

In [99]:
pd.DataFrame(country_list).to_csv('../data/country.csv',index=False)

In [66]:
coffee.head(5)

,name,roaster,roaster_city,roaster_country_state,roaster_country,origin,origin_count,origin_country,review_date,est_price,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,desc
0,“Sweety” Espresso Blend,A.R.C.,Hong Kong,Hong Kong,Hong Kong,Panama; Ethiopia,2,Ethiopia; Panama,November 2017,HKD $250/227 grams,...,9.0,0.0,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric..."
1,Flora Blend Espresso,A.R.C.,Hong Kong,Hong Kong,Hong Kong,Africa; Asia Pacific,2,Africa; Asia Pacific,November 2017,HKD $158/227 grams,...,9.0,0.0,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to..."
2,Ethiopia Shakiso Mormora,Revel Coffee,Billings,Montana,United States,"Guji Zone, southern Ethiopia",1,Ethiopia,November 2017,$16.00/12 ounces,...,9.0,8.0,8.0,9.0,8.0,0.0,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa..."
3,Ethiopia Suke Quto,Roast House,Spokane,Washington,United States,"Guji Zone, Oromia Region, south-central Ethiopia",1,Ethiopia,November 2017,$19.00/16 ounces,...,8.0,8.0,9.0,9.0,8.0,0.0,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"Delicate, sweetly spice-toned. Pink peppercorn..."
4,Ethiopia Gedeb Halo Beriti,Big Creek Coffee Roasters,Hamilton,Montana,United States,"Gedeb District, Gedeo Zone, southern Ethiopia",1,Ethiopia,November 2017,$16.50/12 ounces,...,9.0,9.0,9.0,9.0,8.0,0.0,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"Deeply sweet, subtly pungent. Honey, pear, tan..."


In [9]:
import spacy

nlp = spacy.load("en_core_web_md")

In [10]:
taste_note_input = nlp('juicy')

In [ ]:
similarityValue = []

for i in range(coffee.count()[0]):
    sentence_1 = taste_note_input
    sentence_2 = nlp(coffee.iloc[i][24])
    similarityValue.append(sentence_1.similarity(sentence_2))
#     print(sentence_1, '|', sentence_2, '|', sentence_1.similarity(sentence_2))

coffee['Similarity'] = similarityValue
# print(coffee)

In [ ]:
coffee.loc[:,['name','roaster','roaster_country','origin_country','Similarity']].sort_values(by='Similarity', ascending=False).head(5)

In [ ]:
def tasty_notes_input(words):
    similarityValue = []
    for i in range(coffee.count()[0]):
        sentence_1 = nlp(words)
        sentence_2 = nlp(coffee.iloc[i][24])
        similarityValue.append(sentence_1.similarity(sentence_2))
    coffee['Similarity'] = similarityValue
    return coffee.loc[:,['name','roaster','roaster_country','origin_country',
                         'Similarity']].sort_values(by='Similarity', ascending=False).head(5)

In [ ]:
tasty_notes_input('juicy sweet')

In [ ]:
tasty_notes_input('juicy sweet bright')

In [ ]:
tasty_notes_input('juicy sweet bright fruity')

In [ ]:
def show_recommender(origin, notes):
    coffee_test = coffee[coffee['origin'].str.contains(origin, na=False)]
    similarityValue = []
    for i in range(coffee_test.count()[0]):
        sentence_1 = nlp(notes)
        sentence_2 = nlp(coffee_test.iloc[i][24])
        similarityValue.append(sentence_1.similarity(sentence_2))
    coffee_test['Similarity'] = similarityValue
    return coffee_test.loc[:,['name','roaster','roaster_country','origin_country',
                              'Similarity']].sort_values(by='Similarity', ascending=False).head(5)

In [ ]:
show_recommender(' ', 'juicy sweet bright fruity')

In [31]:
def similarity(coffee1, notes):
    sentence_1 = nlp(notes)
    sentence_2 = nlp(coffee1)
    return sentence_1.similarity(sentence_2)



In [ ]:
r = Parallel(n_jobs=-1, verbose=10, backend="threading")(delayed(query_db)(poi) for poi in list_to_dl[:10])

In [60]:
def show_recommender_2(coffee, notes, no_rec, origin=None):

        
    similarityValue = Parallel(n_jobs=-1, 
                               backend="threading")(delayed(similarity)(coffee1, notes) for coffee1 in coffee['desc'])
#     similarityValue = []
#     for i in range(df.count()[0]):
#         sentence_1 = nlp(notes)
#         sentence_2 = nlp(df.iloc[i][24])
#         similarityValue.append(sentence_1.similarity(sentence_2))

    coffee['Similarity'] = similarityValue
    return coffee.sort_values(by='Similarity', ascending=False).loc[:,['name','roaster','roaster_country',
                                                                   'origin_country','aroma', 'acid', 'body', 
                                                                   'flavor', 'aftertaste']].head(no_rec)


In [34]:
show_recommender_2(coffee, 'juicy sweet bright fruity')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapse

,name,roaster,roaster_country,origin_country,aroma,acid,body,flavor,aftertaste
1742,Ethiopia Washed Sidama Bensa Bombe,Kakalove Cafe,Taiwan,Ethiopia,9.0,9.0,9.0,9.0,8.0
1581,Guatemala Finca Granada Natural,Plat Coffee Roastery,Hong Kong,Guatemala,9.0,8.0,8.0,9.0,8.0
3,Ethiopia Suke Quto,Roast House,United States,Ethiopia,8.0,8.0,9.0,9.0,8.0
1741,Sweet Holiday Blend,Red Rooster Coffee Roaster,United States,Colombia; Costa Rica; Ethiopia; Guatemala,9.0,9.0,9.0,9.0,8.0
1745,Panama El Burro Estate Natural Gesha,Kakalove Cafe,Taiwan,Panama,9.0,9.0,9.0,9.0,9.0
2171,Guatemala Los Santos,Temple Coffee,United States,Guatemala,9.0,9.0,8.0,9.0,8.0
2246,El Triunfo Mexico,JBC Coffee Roasters,United States,Mexico,9.0,8.0,8.0,9.0,8.0
2046,Philippines Sitio Belis 1017,Mostra Coffee,United States,Philippines,9.0,9.0,8.0,9.0,8.0
1736,Ethiopia Tamiru Tadesse Anaerobic,Genesis Coffee Lab,United States,Ethiopia,9.0,9.0,9.0,10.0,9.0
57,Panama Don Julian Pacamara,Willoughby's Coffee & Tea,United States,Panama,9.0,9.0,9.0,9.0,9.0


In [24]:
taste_input = [[5,2,8,5,9]]

In [61]:
def knn_recommender(coffee, taste_input):
    df = coffee.copy()
    taste_feature = df.loc[:,['aroma', 'acid', 'body', 'flavor', 'aftertaste']]
    neighbour_taste = NearestNeighbors(n_jobs=-1)
    neighbour_taste.fit(taste_feature)
    taste_index = neighbour_taste.kneighbors(taste_input, 50, return_distance=False)
    select_taste_index = np.reshape(taste_index,-1)
    return df.iloc[select_taste_index,:]

In [62]:
def main_recommender(coffee, notes, taste_input, origin=None, no_rec=3):
    df = coffee.copy()
    if origin:
        df = df[df['origin'].str.contains(origin, na=False)]
    coffee_filtered = knn_recommender(df, taste_input)    
    final_recommender = show_recommender_2(coffee_filtered, notes, origin=origin, no_rec=no_rec)
    return final_recommender

In [63]:
main_recommender(coffee, 'juicy sweet bright fruity', taste_input)

,name,roaster,roaster_country,origin_country,aroma,acid,body,flavor,aftertaste
1546,Red Clay Espresso Blend,Peach Coffee Roasters,United States,Brazil; Colombia,8.0,0.0,8.0,9.0,9.0
1067,CBD Coffee Dark Roast,Rooster Roastery,United States,Hawaii,7.0,6.0,7.0,7.0,7.0
448,Moonbow Decaf Espresso,Paradise Roasters,United States,South America,8.0,0.0,8.0,8.0,8.0


In [100]:
rec = main_recommender(coffee, 'juicy sweet bright fruity', taste_input)

In [111]:
rec['name'].iloc[0]

'Red Clay Espresso Blend'